In [1]:
import joblib

import numpy as np
import networkx as nx

from src.CBN import CausalBayesianNetwork as CBN
import modularised_utils as mut
import Linear_Additive_Noise_Models as lanm
import operations as ops
import evaluation_utils as evut
import params

import itertools
import joblib
import pickle 
import random

import networkx as nx
import numpy as np
import pandas as pd
import cvxpy as cp

import matplotlib.pyplot as plt
import matplotlib.pylab as pl

from pgmpy import inference

import ot
import ot.plot
from ot.datasets import make_1D_gauss as gauss

import scipy.optimize as optimize
from scipy.spatial.distance import cdist, squareform, pdist
from scipy.stats import wasserstein_distance
from scipy.optimize import linprog
from scipy import stats

from collections import Counter
from IPython.utils import io
from cvxpy.error import SolverError
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from itertools import chain, combinations
from sklearn.linear_model import LinearRegression, Ridge

import numpy as np
from scipy.stats import norm
from pgmpy.models import BayesianNetwork as CBN
from pgmpy.factors.discrete import TabularCPD as cpd

np.random.seed(0)
np.set_printoptions(precision=4,suppress=True)

[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.


In [ ]:
# Example usage:
# 1. Create low-level model
M_STC = M_pgmpy_chain_STC_gaussian()

# 2. Generate some samples from the low-level model
Dll = Dll_samples[None]
# Define the abstraction map
Tau = np.array([[1, 2, 1],  # Maps to high-level state 1 (smoking causes cancer)
              [0, 1, 0]]) # Maps to high-level state 0 (no smoking, no cancer)

# 3. Create high-level model using the abstraction
M_SC = M_pgmpy_chain_SC_from_data(Dll, Tau)

In [16]:
def get_distribution(samples, variables=None):
    
    if variables is None:
        raise ValueError("variables must be specified")
        
    df = (samples.groupby(variables).size() / samples.shape[0]).to_frame()
    return df

def df_to_dict(df):
    d = {}
    for mi, row in df.iterrows():
        d[mi] = row.iloc[0]
    return d

def fillout(dict_, model):

    # Determine the length of the binary tuples
    n = len(list(dict_.keys())[0])
    
    # Generate all possible binary tuples of length n
    #keys = list(itertools.product([0, 1], repeat=n))
    cards = (np.arange(model.get_cardinality(n)) for n in model.nodes)
    keys = list(itertools.product(*cards))
    # Create a new dictionary with all keys from keys list and 0 as the default value
    # If the key exists in the original dictionary, use its value instead of the default
    
    return {key: dict_.get(key, 0) for key in keys}


def create_pairs(n_samples, I_relevant, omega, M_base, M_abst):
    pairs = []
    
    for iota in I_relevant:
        # Skip None values
        with io.capture_output() as captured:
            if iota is None or iota.intervention == {None: None}:
                # Handle the case where there's no intervention
                D_base = M_base.simulate(n_samples=n_samples)
                D_abst = M_abst.simulate(n_samples=n_samples)
            else:
                # Get intervention variables
                intervention_vars = iota.Phi()
                
                # Apply intervention to base model
                D_base = M_base.do(intervention_vars).simulate(
                    n_samples=n_samples, 
                    evidence=iota.vv()
                )
                D_abst = M_abst.do(omega[iota].Phi()).simulate(
                    n_samples=n_samples, 
                    evidence=omega[iota].vv()
                )
        
            # Get distributions
            df_base = get_distribution(D_base, variables=list(M_base.nodes))
            df_abst = get_distribution(D_abst, variables=list(M_abst.nodes))

            p_base = fillout(df_to_dict(df_base), M_base)
            p_abst = fillout(df_to_dict(df_abst), M_abst)
                
            pairs.append(ops.Pair(p_base, p_abst, iota, omega[iota]))
            
    return pairs

In [31]:
pairs = create_pairs(num_llsamples, Ill_relevant, omega, M_STC, M_SC)

In [52]:
def generate_omega_cost_matrix(pairs):
    
    lst_base = pairs[0].base_labels
    lst_abst = pairs[0].abst_labels
    df = pd.DataFrame(0, index=lst_abst, columns=lst_base)
    print(df)
    for pair in pairs:
        p = pair.get_domain('base')
        q = pair.get_domain('abst')
        df.loc[q, p] = df.loc[q, p] - 1
    cost_matrix = df.values.astype(float)  # convert to float array
    
    reg = np.abs(np.min(cost_matrix))
    cost_matrix += reg
    cost_matrix +=  np.ones_like(cost_matrix)
    
    return cost_matrix

In [68]:
generate_omega_cost_matrix(pairs, Ill_relevant)

AttributeError: 'NoneType' object has no attribute 'get_variable'

In [53]:
pairs[0].base_labels

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [48]:
generate_omega_cost_matrix(pairs)

TypeError: 'Pair' object is not subscriptable

In [37]:
class Poset: 
    def __init__(self, data):
        self.data = data
        self.children = []

    def add_child(self, child):
        self.children.append(child)

    def to_path(self, current_path):
        path = current_path[:]
        path.append(self)
        return path
    
    def get_child(self, current_path):
        for i in range(len(current_path) - 1):
            if current_path[i] == self:
                return current_path[i + 1]
        return None

In [38]:
def to_chains(pairs, tree_structure):
    
    plans = [Poset(pair) for pair in pairs]
    
    # Initialize parent-child relations
    for parent_index, child_index in tree_structure:
        plans[parent_index].add_child(plans[child_index])
    
    # Initialize variables
    paths = []
    current_path = []

    def traverse_paths(node):
        nonlocal current_path

        current_path.append(node)

        if not node.children:
            paths.append(current_path[:])

        for i, child in enumerate(node.children):
            traverse_paths(child)

        current_path.pop()

    # Traverse paths starting from each root node
    for plan in plans:
        if not any(node for node in plans if plan in node.children):
            traverse_paths(plan)

    return paths


def drop1omega(omega, key):
    modified_omega = omega.copy()
    if key in modified_omega:
        del modified_omega[key]
    return modified_omega

def interventional_order(iota_i, iota_j):
    i_variables = set(iota_i.intervention.keys())
    j_variables = set(iota_j.intervention.keys())

    if None in i_variables and not None in j_variables:
        return True
    elif i_variables <= j_variables and all(iota_i.intervention[var] == iota_j.intervention[var] for var in i_variables):
        return True
    else:
        return False
    
def build_poset(Iota):
    
    edges = []
    tree = {}

    for i in Iota:
        tree[i] = []
    
    for i in range(len(Iota)):
        for j in range(i + 1, len(Iota)):
            if interventional_order(Iota[i], Iota[j]):
                edges.append((i, j))
                tree[Iota[i]].append(Iota[j])
    
    return edges, tree

In [39]:

struc, tree = build_poset(Ill_relevant[1:])
chains      = to_chains(pairs, struc)

In [43]:
for chain in chains:
    print(chain)
    print( )




[<__main__.Poset object at 0x17c29b020>, <__main__.Poset object at 0x17c29a7b0>]




### Define low-level DCM "LL" and high-level DCM "HL"

In [3]:
experiment = 'synth1'

In [4]:
S = 'Smoking'
T = 'Tar'
C = 'Cancer'

S_ = 'Smoking_'
C_ = 'Cancer_'

In [5]:
ll_endogenous_coeff_dict = {(S, T): 0.3, (T, C): 0.2}
#ll_endogenous_coeff_dict = {(S, T): 1.0, (T, C): .3} 
#ll_endogenous_coeff_dict = {(S, T): 4.0, (T, C): 3} 
#ll_endogenous_coeff_dict = {(S, T): 10.0, (T, C): 8.0} 

ll_causal_graph          = CBN(list(ll_endogenous_coeff_dict.keys()))
#nx.draw(nx.DiGraph(ll_causal_graph.edges()),with_labels=True)

hl_endogenous_coeff_dict = {(S_, C_): 0.0}
hl_causal_graph          = CBN(list(hl_endogenous_coeff_dict.keys()))
# #nx.draw(nx.DiGraph(hl_causal_graph.edges()),with_labels=True)

In [6]:
# Define the number of samples from the low-level environment.
num_llsamples   = params.n_samples['synth1'][0]
num_hlsamples   = params.n_samples['synth1'][1]

### Construct the empirical nominal distribution for the low-level model

In [7]:
ll_mu_hat    = np.array([0, 0, 0])  
ll_Sigma_hat = np.diag([1, 1, 1])  #np.diag([1, 2, 1]) 

hl_mu_hat    = np.array([0, 0]) 
hl_Sigma_hat = np.diag([1, 1]) #np.diag([1, 2])

### Define the sets of relevant interventions and the (total) surjective and order-preserving function $ω:I^{L} \mapsto I^{H}$

In [8]:
iota0 = None
iota1 = ops.Intervention({S:0})
iota2 = ops.Intervention({S:0, T:1})
iota3 = ops.Intervention({S:1})
iota4 = ops.Intervention({S:1, T:0})
iota5 = ops.Intervention({S:1, T:1})

eta0 = None
eta1 = ops.Intervention({S_:0})
eta2 = ops.Intervention({S_:1})

omega = {   
            iota0: eta0,
            iota1: eta1,
            iota2: eta1,
            iota3: eta2,
            iota4: eta2,
            iota5: eta2
        }

Ill_relevant = list(set(omega.keys()))
Ihl_relevant = list(set(omega.values()))

### Sampling and Pair construction

In [9]:
Dll_samples, Dll_noise = {}, {}
for iota in Ill_relevant:

    llcm              = lanm.LinearAddSCM(ll_causal_graph, ll_endogenous_coeff_dict, iota)
    #Different Dll_noise for each iota
    lenv_iota         = mut.sample_distros_Gelbrich([(ll_mu_hat, ll_Sigma_hat)])[0] 
    Dll_noise[iota]   = lenv_iota.sample(num_llsamples)[0]
    Dll_samples[iota] = llcm.simulate(Dll_noise[iota], iota)

### Define the abstraction T

In [9]:
#T = np.array([[.1, .6, .2], [.6, .2, .7]])
T = np.array([[1, 2, 1], [0, 1, 0]])

### Compute the empirical nominal distribution for the high-level model and the linear coefficients

In [10]:
data_observational_hl         = Dll_samples[None]@ T.T
hl_endogenous_coeff_dict      = mut.get_coefficients(data_observational_hl, hl_causal_graph) 
U_hl, hl_mu_hat, hl_Sigma_hat = mut.lan_abduction(data_observational_hl, hl_causal_graph, hl_endogenous_coeff_dict)

In [11]:
# data_observational_hl    = Dll_samples[None]@ T.T
# U_hl                     = np.random.multivariate_normal(mean=hl_mu_hat, cov=hl_Sigma_hat, size=num_hlsamples)
# hl_endogenous_coeff_dict = mut.get_coefficients_with_known_noise(data_observational_hl, U_hl, hl_causal_graph)

### Generate samples for the high-level model

In [12]:
Dhl_samples, Dhl_noise = {}, {}
for eta in Ihl_relevant:

    if eta is not None:
        hlcm             = lanm.LinearAddSCM(hl_causal_graph, hl_endogenous_coeff_dict, eta)
        #Different Dll_noise for each iota
        lenv_eta         = mut.sample_distros_Gelbrich([(hl_mu_hat, hl_Sigma_hat)])[0] 
        Dhl_noise[eta]   = lenv_eta.sample(num_hlsamples)[0]
        Dhl_samples[eta] = hlcm.simulate(Dhl_noise[eta], eta)

    else:
        Dhl_noise[eta]   = U_hl
        Dhl_samples[eta] = data_observational_hl

In [13]:
# # Closer to exact abstraction
# Dhl_samples = {}
# list_of_iotas = []
# for iota in Ill_relevant:
    
#     if iota not in list_of_iotas:
#         list_of_iotas.append(omega[iota])
#         if iota is not None:
#             Dhl_samples[omega[iota]] = Dll_samples[iota] @ T.T
#         else:
#             Dhl_samples[omega[iota]] = data_observational_hl

In [14]:
Ds = {}
for iota in Ill_relevant:
    Ds[iota] = (Dll_samples[iota], Dhl_samples[omega[iota]])

### Save the data

In [15]:
joblib.dump((ll_causal_graph, Ill_relevant), f"data/{experiment}/LL.pkl")
joblib.dump(ll_endogenous_coeff_dict, f"data/{experiment}/ll_coeffs.pkl")

joblib.dump((hl_causal_graph, Ihl_relevant), f"data/{experiment}/HL.pkl")
joblib.dump(hl_endogenous_coeff_dict, f"data/{experiment}/hl_coeffs.pkl")

joblib.dump(Ds, f"data/{experiment}/Ds.pkl")

joblib.dump(T, f"data/{experiment}/Tau.pkl")
joblib.dump(omega, f"data/{experiment}/omega.pkl")

joblib.dump((Dll_noise[None], ll_mu_hat, ll_Sigma_hat), f"data/{experiment}/exogenous_LL.pkl")
joblib.dump((U_hl, hl_mu_hat, hl_Sigma_hat), f"data/{experiment}/exogenous_HL.pkl")

['data/synth1/exogenous_HL.pkl']